# Dog Breeds classification PyTorch

## Loading dataset

In [4]:
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

In [5]:
batch_size = 20

In [6]:
# data_transforms = {
#     'train': transforms.Compose([
#         transforms.Resize(224),
#         transforms.ToTensor(),
#         transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
#     ]),
#     'valid': transforms.Compose([
#         transforms.Resize(224),
#         transforms.ToTensor(),
#         transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
#     ]),
#     'test': transforms.Compose([
#         transforms.Resize(224),
#         transforms.ToTensor(),
#         transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
#     ]),
# }

# we don't randomize data as it is already random enough
data_transforms = transforms.Compose([
    transforms.Resize(224),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [7]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

image_datasets = {
    'train': ImageFolder(root='dogImages/train', transform=data_transforms),
    'valid': ImageFolder(root='dogImages/valid', transform=data_transforms),
    'test':  ImageFolder(root='dogImages/test',  transform=data_transforms)
}

In [8]:
data_loaders = {
    'train': DataLoader(image_datasets['train'], batch_size=batch_size, shuffle=True),
    'valid': DataLoader(image_datasets['valid'], batch_size=batch_size),
    'test':  DataLoader(image_datasets['test'],  batch_size=batch_size)
}

In [9]:
print('Train images', len(image_datasets['train']))
print('Valid images', len(image_datasets['valid']))
print('Test images', len(image_datasets['test']))
dog_classes = len(image_datasets['train'].classes)
print('Dog classes', dog_classes)
steps_per_epoch = len(image_datasets['train'])/20
print('Epoch steps', steps_per_epoch)

Train images 6680
Valid images 835
Test images 836
Dog classes 133
Epoch steps 334.0


In [10]:
import torch.nn as nn

In [11]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        #defining NN layers
        self.seq = nn.Sequential(
            #input: 3x224x224
            nn.Conv2d(in_channels=3, out_channels=16, kernel_size=2), #16x223x223
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.MaxPool2d(4), #16x56x56
            nn.Dropout(0.2),
            
            nn.Conv2d(in_channels=16, out_channels=32, kernel_size=2), #32x55x55
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.MaxPool2d(4), #32x14x14
            nn.Dropout(0.2),
            
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=2), #64x13x13
            nn.ReLU(),
            nn.BatchNorm2d(64),
            nn.MaxPool2d(4), #64x3x3
            nn.Dropout(0.2),
            
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=2), #128x2x2
            nn.ReLU(),
            nn.BatchNorm2d(128),
            nn.Flatten(start_dim=1), # 128x2x2 = 512
            nn.Dropout(0.25),
            
            nn.Linear(in_features=128*2*2, out_features=1024), #512
            nn.ReLU(),
            nn.Dropout(0.5),
            
            nn.Linear(in_features=1024, out_features=dog_classes),
            nn.Softmax()  
        )
    
    def forward(self, x):
        return self.seq(x)

In [12]:
model = Net()

In [13]:
import torch

use_cuda = torch.cuda.is_available()
if use_cuda:
    model.cuda()

In [15]:
if use_cuda:
    model.load_state_dict(torch.load('pytorch_dogs_class.pt'))
else:
    model.load_state_dict(torch.load('pytorch_dogs_class.pt', map_location=torch.device('cpu')))

In [20]:
import numpy as np
from tqdm.notebook import tqdm

def train(n_epochs, loaders, model, optimizer, criterion, save_path):
    valid_loss_min = np.Inf
    
    for epoch in range(1, n_epochs+1):
        train_loss = 0.0
        valid_loss = 0.0
        
        model.train()
        for batch_idx, (data, labels) in tqdm(enumerate(loaders['train']), total=steps_per_epoch):
            if use_cuda:
                data, labels = data.cuda(), labels.cuda()
            
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, labels)
            loss.backward()
            optimizer.step()
            train_loss += (1 / (batch_idx + 1)) * (loss.data - train_loss)
            
        model.eval()
        for batch_idx, (data, labels) in enumerate(loaders['valid']):
            if use_cuda:
                data, labels = data.cuda(), labels.cuda()
                
            output = model(data)
            loss = criterion(output, labels)
            valid_loss += (1 / (batch_idx + 1)) * (loss.data - valid_loss)
            
        train_loss /= len(loaders['train'].dataset)
        valid_loss /= len(loaders['valid'].dataset)
        
        print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
            epoch, 
            train_loss,
            valid_loss
        ))
        
        if valid_loss <= valid_loss_min:
            print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
                valid_loss_min,
                valid_loss
            ))
            torch.save(model.state_dict(), save_path)
            valid_loss_min = valid_loss
            
    return model

In [1]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())

NameError: name 'nn' is not defined

In [31]:
epochs = 5

train(epochs, data_loaders, model, optimizer, criterion, 'pytorch_dogs_class.pt')


Epoch: 1 	Training Loss: 0.000712 	Validation Loss: 0.005770
Validation loss decreased (inf --> 0.005770).  Saving model ...



Epoch: 2 	Training Loss: 0.000712 	Validation Loss: 0.005772



Epoch: 3 	Training Loss: 0.000712 	Validation Loss: 0.005764
Validation loss decreased (0.005770 --> 0.005764).  Saving model ...



Epoch: 4 	Training Loss: 0.000711 	Validation Loss: 0.005782



Epoch: 5 	Training Loss: 0.000711 	Validation Loss: 0.005754
Validation loss decreased (0.005764 --> 0.005754).  Saving model ...


Net(
  (seq): Sequential(
    (0): Conv2d(3, 16, kernel_size=(2, 2), stride=(1, 1))
    (1): ReLU()
    (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): MaxPool2d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
    (4): Dropout(p=0.2, inplace=False)
    (5): Conv2d(16, 32, kernel_size=(2, 2), stride=(1, 1))
    (6): ReLU()
    (7): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): MaxPool2d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
    (9): Dropout(p=0.2, inplace=False)
    (10): Conv2d(32, 64, kernel_size=(2, 2), stride=(1, 1))
    (11): ReLU()
    (12): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (13): MaxPool2d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
    (14): Dropout(p=0.2, inplace=False)
    (15): Conv2d(64, 128, kernel_size=(2, 2), stride=(1, 1))
    (16): ReLU()
    (17): BatchNorm2d(128, eps=

In [16]:
def test(loaders, model, criterion):
    test_loss = 0.0
    correct = 0.0
    total = 0.0
    
    model.eval()
    
    for batch_idx, (data, labels) in enumerate(loaders['test']):
        if use_cuda:
            data, labels = data.cuda(), labels.cuda()

        output = model(data)
        loss = criterion(output, labels)
        test_loss += (1 / (batch_idx + 1)) * (loss.data - test_loss)
        pred = output.data.max(1, keepdim=True)[1]
        correct += np.sum(np.squeeze(pred.eq(labels.data.view_as(pred))).cpu().numpy())
        total += data.size(0)
        
    test_loss /= len(loaders['test'].dataset)
            
    print('Test Loss: {:.6f}\n'.format(test_loss))
    
    print('\nTest Accuracy: %2d%% (%2d/%2d)' % (
        100. * correct / total, correct, total))
    

In [21]:
test(data_loaders, model, criterion)

Test Loss: 0.005744


Test Accuracy: 10% (85/836)
